In [5]:
import numpy as np
import torch
import sympy


In [ ]:
class BaseLayer:
    def update(self,eta):
        self.w -= eta * self.grad_w
        self.b -= eta * self.grad_b 

class MiddleLayer(BaseLayer):
    def __init__(self, n_upper, n):
        self.w = np.random.randn(n_upper, n) * np.sqrt(2/n_upper)  # Heの初期値
        self.b = np.zeros(n)

    def forward(self, x):
        self.x = x
        self.u = np.dot(x, self.w) + self.b
        self.y = np.where(self.u <= 0, 0, self.u) # ReLU
    
    def backward(self, grad_y):
        delta = grad_y * np.where(self.u <= 0, 0, 1)

        self.grad_w = np.dot(self.x.T, delta)
        self.grad_b = np.sum(delta, axis=0)     
        self.grad_x = np.dot(delta, self.w.T) 

class GenOutLayer(BaseLayer):
    def __init__(self, n_upper, n):
        self.w = np.random.randn(n_upper, n) / np.sqrt(n_upper)
        self.b = np.zeros(n)
    
    def forward(self, x):
        self.x = x
        u = x@self.w
        self.y = np.tanh(u)

    def backward(self, grad_y):
        delta = grad_y * (1-self.y**2)

        self.grad_w = self.x.T@delta
        self.grad_b = np.sum(delta, axis = 0)
        self.grad_x = delta@self.w.T

class DiscOutLayer(BaseLayer):
    def __init__(self,n_upper,n):
        self.w = np.random.randn(n_upper,n)
        self.b = np.zeros(n)
    
    def forward(self, x):
        self.x = x
        u = x@self.w
        self.y = 1/(1+np.exp(-u))
    
    def backward(self, t):
        delta = self.y - t

        self.grad_w = self.x.T@delta
        self.grad_b = np.sum(delta, axis = 0)
        self.grad_x = delta@self.w.T


In [ ]:
#順伝播と逆伝播実装

gen_layers = [MiddleLayer(n_noise, 32),
             Middle(32,64)
             GenOutLayeer(64,img_size*img_size)]

disc_layers = [MiddleLayer(img_size*img_size, 64),
               MiddleLayer(64, 32),
               DiscOutLayer(32, 1)]

#自分で順伝播、逆伝播書こうのコーナー
#順伝播
def forward_propagation(x, layers):
    for layer in layers:
        layer.forward(x)
        x = layer.x
    return x

#逆伝播
def backpropagation(t):
    grad_y = t
    for layer in reversed(layers, layers):
        layer.backward(grad_y)
        grad_y = layer.grad_x
    return grad_y

#訓練
def train_model(x, t, prop_layers, update_layers):
    y = forward_propagation(x, prop_layers)
    backpropagation(t, prop_layers)
    update_params(update_layers)
    return (get_error(y, t), get_accuracy(y, t))


In [ ]:
batch_half = batch_size // 2
error_record = np.zeros((n_learn, 2))
acc_record = np.zeros((n_learn,2))

for i in range(n_learn):
    #Gで生成したもので、Dの訓練
    #ノイズの作成
    noise = np.random.rormal(0, 1, (batch_half, n_noise))#平均0分散1turple二次元の配列ndarrayを用意
    imgs_fake = forward_propagation(noise, gen_layers)#ノイズを基にGで生成
    t = np.zeros((batch_half, 1))#正解（今回は全て偽なので0のみ）
    error, accuracy = train_model(imgs_fake, t, disc_layers, disc_layers)#Dに入力し訓練、更新
    #誤差、正解率の記録
    error_record[i][0] = error
    acc_record[i][0] = accuracy

    #本物でDを訓練
    rand_ids = np.random.randint(len(x_train), size = batch_half)#0からlen(x_train)までの整数値を指定サイズで返す
    imgs_real = x_train[rand_ids, :]
    t = np.ones((batch_half, 1))
    error, accuracy = train_model(imds_real, t, disc_layers, disc_layers)

    error_record[i][1] = error
    acc_record[i][1] = accuracy

    #Gの訓練
    noise = np.random.normal(0, 1, (batch_size, n_noise))
    t = np.ones((batch_size, 1))
    train_model(noise, t, gen_layers + disc_layers, gen_layers)

In [13]:
#画像の生成
def generate_images(i):
    n_rows = 16#全体に並べるときの列数
    n_cols = 16
    noise = np.random.normal(0, 1, (n_rows*n_cols, n_noise))
    g_imgs = forward_propagation(noise, gen_layers)
    g_imgs = g_imgs/2 + 0.5#tanhで出力されたものをスケーリング、シフトして0～1に収める→matplotlibで適した形に

    img_size_spaced = img_size + 2 #各画像の間にスペースを空けるため
    matrix_image = np.zeros((img_size_spaced*n_rows, img_size_spaced*n_cols)) #画像を張るための場所を作成

    #生成された画像を表示
    for r in range(n_rows):
        for c in range(n_cols):
            g_img = g_imgs[r*n_cols + c].reshape(img_size, img_size)
            #画像の始まる場所を指定(高さ)
            top = r * img_size_spaced
            left = c * img_size_apaced
            matrix_image[top:top+img_size, left:left:left+img_size]
    
    plt.figure(figsize=(8,8))
    plt.imshow(matrix_image.tolist(), cmap = "Greys_r")
    plt.tick_params(labelbottom = False, labelleft = False, bottom = False, left = False)
    plt.show()


